In [ ]:
#reload the modules before running the program
%load_ext autoreload
%autoreload 2

import numpy as np
import scipy
from scipy import signal
from scipy.fft import fftshift
from scipy.fft import fft, fftfreq

import matplotlib.pyplot as plt
import matplotlib as mpl

from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

import os
import math
import cv2

In [ ]:
from datasets import RFData, CameraData, SignalDataset, FuseDatasets
import metrics
import utils

In [ ]:
np.random.seed(0)
torch.manual_seed(0)
data_path = r"D:\osa_new_proc_rev"
# trial_folders = ["v_12_5", "v_13_5", "v_13_10"]
trial_folders = os.listdir(data_path)

# train_idxs = np.linspace(0, 49, 50, dtype=int)
train_folders = trial_folders#[trial_folders[i] for i in train_idxs]
# test_folders = [trial_folders[i] for i in range(len(trial_folders)) if i not in train_idxs]


thermal_file_name = "Thermal_Camera"
num_samps_oversample = None # per experiment, number of samples to generate
data_length = 9000
fs = 30
out_len = 9000 # sample length generated
thermal_ext = ".npy"

dataset_thermal_train = CameraData(data_path, train_folders, thermal_file_name, num_samps_oversample, fs, data_length, out_len, thermal_ext)
# dataset_thermal_test = CameraData(data_path, test_folders, thermal_file_name, num_samps_oversample, fs, data_length, out_len, thermal_ext)

In [ ]:
thermal_vids = []
dataset_thermal_train.get_vid = True
for i in tqdm(range(len(dataset_thermal_train))):
    # print(i)
    thermal = dataset_thermal_train.__getitem__(i)
    # frame = thermal[0,:,:]
    thermal_vids.append(thermal)

np.save(r"F:\OSA_local_version\OSA_project\Emir_Clean\saved_data9000\thermal_vids.npy", np.array(thermal_vids))
dataset_thermal_train.get_vid = False

In [ ]:
plt.plot(dataset_thermal_train.__getitem__(0))

In [ ]:
utils.save_video(dataset_thermal_train, 3280, path=r"F:\OSA_local_version\OSA_project")

In [ ]:
dataset_thermal_train.trial_folders[dataset_thermal_train.oversampling_idxs[3280][0]] 

In [ ]:
samp_f=5e6
freq_slope=60.012e12
samples_per_chirp=256
num_tx = 3
num_rx = 4
radar_file_name = "FMCW_Radar.npy"
window_size = 15 # number of range bins to use

dataset_radar_train = RFData(data_path, train_folders, data_length, radar_file_name, out_len, window_size, samples_per_chirp, samp_f, freq_slope, num_samps_oversample, num_tx, num_rx, fs)
# dataset_radar_test = RFData(data_path, test_folders, data_length, radar_file_name, out_len, window_size, samples_per_chirp, samp_f, freq_slope, num_samps_oversample, num_tx, num_rx, fs)

In [ ]:
np.random.seed(0)
torch.manual_seed(0)
vital_dict_file_name = "gt_dict.pkl"
vital_key_radar = "CHEST"
vital_key_thermal = "AIR_flow"
l_freq_bpm = 5
u_freq_bpm = 30
dataset_OSA_train = SignalDataset(data_path, train_folders, vital_dict_file_name, 'OSA', data_length, out_len, False, fs, 1024, False, l_freq_bpm, u_freq_bpm, num_samps_oversample, False,
                              normalize=False)
dataset_CSA_train = SignalDataset(data_path, train_folders, vital_dict_file_name, 'CSA', data_length, out_len, False, fs, 1024, False, l_freq_bpm, u_freq_bpm, num_samps_oversample, False,
                              normalize=False)
dataset_MSA_train = SignalDataset(data_path, train_folders, vital_dict_file_name, 'MSA', data_length, out_len, False, fs, 1024, False, l_freq_bpm, u_freq_bpm, num_samps_oversample, False,
                              normalize=False)
dataset_hypopnea_train = SignalDataset(data_path, train_folders, vital_dict_file_name, 'Hypopnea', data_length, out_len, False, fs, 1024, False, l_freq_bpm, u_freq_bpm, num_samps_oversample, False,
                              normalize=False)
dataset_partial_apnea_train = SignalDataset(data_path, train_folders, vital_dict_file_name, 'Partial_Apnea', data_length, out_len, False, fs, 1024, False, l_freq_bpm, u_freq_bpm, num_samps_oversample, False,
                              normalize=False)
dataset_gt_thermal_train = SignalDataset(data_path, train_folders, vital_dict_file_name, vital_key_thermal, data_length, out_len, False, fs, 1024, False, l_freq_bpm, u_freq_bpm, num_samps_oversample, False)
dataset_gt_radar_train = SignalDataset(data_path, train_folders, vital_dict_file_name, vital_key_radar, data_length, out_len, True, fs, 1024, False, l_freq_bpm, u_freq_bpm, num_samps_oversample, False)

# dataset_apnea_test = SignalDataset(data_path, test_folders, vital_dict_file_name, 'Sleep_Apnea', data_length, out_len, False, fs, 1024, False, l_freq_bpm, u_freq_bpm, num_samps_oversample, False,
#                               normalize=False)
# dataset_gt_thermal_test = SignalDataset(data_path, test_folders, vital_dict_file_name, vital_key_thermal, data_length, out_len, False, fs, 1024, False, l_freq_bpm, u_freq_bpm, num_samps_oversample, False)
# dataset_gt_radar_test = SignalDataset(data_path, test_folders, vital_dict_file_name, vital_key_radar, data_length, out_len, True, fs, 1024, False, l_freq_bpm, u_freq_bpm, num_samps_oversample, False)

In [ ]:
np.random.seed(0)
torch.manual_seed(0)
fused_dataset_train = FuseDatasets([dataset_radar_train, dataset_thermal_train, dataset_gt_radar_train, dataset_gt_thermal_train, dataset_OSA_train, dataset_CSA_train, dataset_hypopnea_train, dataset_partial_apnea_train], ["radar", "thermal", "gt_radar", "gt_ir", "gt_OSA", "gt_CSA", "hypopnea", "partial_apnea"], out_len=out_len)
# fused_dataset_test = FuseDatasets([dataset_radar_test, dataset_thermal_test, dataset_gt_radar_test, dataset_gt_thermal_test, dataset_apnea_test], ["radar", "thermal", "gt_radar", "gt_ir", "gt_apnea"], out_len=out_len)
t_arr = np.linspace(0, out_len/fs, out_len)

In [ ]:
patient_idx_container = utils.get_patient_idxs(dataset_thermal_train, patient_ids=['v_12', 'v_13', 'v_20', 'v_29', 'v_30', 'v_32', 'v_36', 'v_37', 'v_39', 'v_40'])

In [ ]:
# Create a folder to save the index arrays
folder_path = 'F:\OSA_local_version\OSA_project\Emir_Clean\patient_folder9000'
os.makedirs(folder_path, exist_ok=True)
patient_ids = ['v_12', 'v_13', 'v_20', 'v_29', 'v_30', 'v_32', 'v_36', 'v_37', 'v_39', 'v_40']
for i, patient_id in enumerate(patient_ids):
    patient_num = patient_id.split('_')[1]
    idx_path = os.path.join(folder_path, f'patient_idxs_{patient_num}.npy')
    np.save(idx_path, np.array(patient_idx_container[i]))

In [ ]:
np.random.seed(0)
torch.manual_seed(0)
dataset = []
gt_sigs = []
for i in tqdm(range(len(fused_dataset_train))):
    thermal = fused_dataset_train[i]['thermal']
    thermal = utils.highpass_filter(thermal, fs=30, fc=0.1/2, order=6)
    # thermal = signal.savgol_filter(thermal, 41, 3)
    thermal = (thermal - np.mean(thermal))/np.std(thermal)
    dataset.append(thermal)
    gt_sigs.append(np.array([fused_dataset_train[i]['gt_ir'][:,0], fused_dataset_train[i]['gt_OSA'][:,0], fused_dataset_train[i]['gt_CSA'][:,0], fused_dataset_train[i]['gt_radar'][:,0], fused_dataset_train[i]['hypopnea'][:,0]]))

In [ ]:
path = r"F:\OSA_local_version\OSA_project\Emir_Clean\saved_data9000"
os.makedirs(path, exist_ok=True)
np.save(os.path.join(path, r"thermal_arr.npy"), dataset)
np.save(os.path.join(path, r"gt_arr.npy"), gt_sigs)

In [ ]:
radar_arr = []
for i in tqdm(range(len(fused_dataset_train))):
    radar_data = fused_dataset_train[i]['radar']
    power = []
    filt_s = []
    for k in range(15):
        phase_signal = utils.unwrap_phase(radar_data[:, 0, 0, :, k])
        filtered_x = utils.highpass_filter(phase_signal, order=6, fc=0.1/2, plot=False)
        # filtered_x = signal.savgol_filter(filtered, 41, 3)
        filtered_x = (filtered_x - np.mean(filtered_x)) / np.std(filtered_x)
        filt_s.append(filtered_x)

        f, Pxx_spec = signal.welch(filtered_x, fs, 'flattop', 1024, scaling='spectrum')
        power.append(np.sqrt(Pxx_spec.max()))
    radar_arr.append(filt_s[np.argmax(power)])


In [ ]:
np.save(os.path.join(path, r"radar_arr.npy"), radar_arr)
# radar_arr = np.load(r"E:\OSA_project\Emir_Clean\saved_idx\radar_arrv5.npy")

In [ ]:
path

In [ ]:
power = []
filt_s = []
for i in range(25):

        filtered = utils.highpass_filter(utils.normalize_signal(utils.unwrap_phase(radar_data[0,:, 0, 0, :, i].numpy())), order=6, fc=0.1/2, plot=False)
        filtered_x = signal.savgol_filter(filtered, 41, 3)
        filt_s.append(filtered_x)

        f, Pxx_spec = signal.welch(filtered_x, fs, 'flattop', 1024, scaling='spectrum')
        # plt.figure()
        # plt.semilogy(f, np.sqrt(Pxx_spec))
        # plt.xlabel('frequency [Hz]')
        # plt.ylabel('Linear spectrum [V RMS]')
        # plt.show()
        print(np.sqrt(Pxx_spec.max()))
        power.append(np.sqrt(Pxx_spec.max()))
        print(i)
        plt.plot(filtered_x, label='filtered')
        plt.plot(gt_data[0,:,0], label='gt')
        plt.legend()
        plt.show()
        print(r"##########################################")
print(np.argmax(power))
plt.figure()
plt.plot(filt_s[np.argmax(power)])
plt.plot(gt_data[0,:,0], label='gt')
plt.legend()
plt.show()

In [ ]:
apnea_idx = []
count = 0
for i, data in tqdm(enumerate(fused_dataset_train)):
    # print(i)
    try:
        batch = data['gt_CSA'][:,0]

        if(np.mean(batch) > 0.1):
            apnea_idx.append(i)
            # patient_id = fused_dataset_train.trial_folders[fused_dataset_train.oversampling_idxs[i][0]]
            # print(patient_id)
    except:
        count = count + 1
        pass

print(100*len(apnea_idx)/len(fused_dataset_train))
print(count, len(apnea_idx))

In [ ]:
NUKS_idx = []

for i, data in tqdm(enumerate(fused_dataset_train)):
    # print(i)
    thermal = data['thermal']
    thermal = (thermal - np.mean(thermal))/np.std(thermal)
    if(utils.detect_NUKS(thermal) == True):
        NUKS_idx.append(i)

print(100*len(NUKS_idx)/len(fused_dataset_train))
print(len(NUKS_idx))

In [ ]:
np.save(os.path.join(path, r"NUKS_idx.npy"), NUKS_idx)
# NUKS_idx = np.load(r"E:\OSA_project\Emir_Clean\saved_idx\NUKS_idxv4.npy")

In [ ]:
mode_lock_idx = []
dataset_thermal_train.get_vid = True
for i in tqdm(range(len(dataset_thermal_train))):
    # print(i)
    thermal = dataset_thermal_train.__getitem__(i)
    # frame = thermal[0,:,:]
    for f_id in range(len(thermal)):
        frame = thermal[f_id,:,:]
        if((frame == frame[0,0]).all() == True):
            mode_lock_idx.append(i)
            break

print(100*len(mode_lock_idx)/len(dataset_thermal_train))
print(len(mode_lock_idx))
dataset_thermal_train.get_vid = False

In [ ]:
np.save(os.path.join(path, r"mode_lock_idx.npy"), mode_lock_idx)
# mode_lock_idx = np.load(r"E:\OSA_project\Emir_Clean\saved_idx\mode_lock_idxv4.npy")

In [ ]:
len(NUKS_idx), len(mode_lock_idx)

In [ ]:
count = 0
for i in range(len(NUKS_idx)):
    if(NUKS_idx[i] in mode_lock_idx):
        count = count + 1
print(count)

In [ ]:
len(fused_dataset_train)

In [ ]:
np.random.seed(0)
torch.manual_seed(0)
movement_idx = []
vid_movement_idx = []
for i in tqdm(range(len(fused_dataset_train))):
    if(i in NUKS_idx):
        # print("NUKS")
        pass
    elif(i in mode_lock_idx):
        # print('mode lock')
        pass
    else:
        thermal = fused_dataset_train[i]['thermal']
        thermal = utils.highpass_filter(thermal, fs=30, fc=0.1, order=2)
        # thermal = signal.savgol_filter(thermal, 41, 3)
        thermal = (thermal - np.mean(thermal))/np.std(thermal)

        dists_min, dists_max, lmin, lmax = utils.movement_detector_radar(thermal, dmin=29, dmax=31, gt_breathing=fused_dataset_train[i]['gt_ir'][:,0], plot=False)
        # print(i)
        if((max(dists_min) > 4.5) or (max(dists_max) > 4.5)):
            movement_idx.append(i)
            # print('idx:', i, 'movement')
        # dataset_thermal_train.get_vid = True
        # if(utils.vid_movement_detector(dataset_thermal_train.__getitem__(i), lb=0, ub=200) == True):
        #     vid_movement_idx.append(i)
        #     print('idx:', i), 'video movement'
        # dataset_thermal_train.get_vid = False


In [ ]:
np.save(os.path.join(path,r"movement_idx.npy"), movement_idx)

In [ ]:
for i in tqdm(range(len(fused_dataset_train))):
    if(i in NUKS_idx):
        # print("NUKS")
        pass
    elif(i in mode_lock_idx):
        # print('mode lock')
        pass
    else:
        dataset_thermal_train.get_vid = True
        if(utils.vid_movement_detector(dataset_thermal_train.__getitem__(i), lb=0, ub=200) == True):
            vid_movement_idx.append(i)
            # print('idx:', i), 'video movement'
        dataset_thermal_train.get_vid = False

In [ ]:
len(movement_idx), len(vid_movement_idx)

In [ ]:
np.save(os.path.join(path, r"vid_movement_idx.npy"), vid_movement_idx)

In [ ]:
for idx in vid_movement_idx:
    if(idx not in movement_idx):
        print(idx) 
    thermal = fused_dataset_train[idx]['thermal']
    thermal = utils.highpass_filter(thermal, fs=30, fc=0.1, order=2)
    # thermal = signal.savgol_filter(thermal, 41, 3)
    thermal = (thermal - np.mean(thermal))/np.std(thermal)
    plt.plot(t_arr, fused_dataset_train[idx]['gt_ir'][:,0])
    plt.plot(t_arr, fused_dataset_train[idx]['gt_CSA'][:,0], linewidth=7.0)
    plt.plot(t_arr, fused_dataset_train[idx]['gt_OSA'][:,0], linewidth=7.0)

    dists_min, dists_max, lmin, lmax = utils.movement_detector_radar(thermal, dmin=29, dmax=31, gt_breathing=fused_dataset_train[i]['gt_ir'][:,0], plot=True)


In [ ]:
len(movement_idx)

In [ ]:
np.save(r"E:\OSA_project\Emir_Clean\saved_idx\movement_idxv5.npy", movement_idx)

In [ ]:
path = r"F:\OSA_local_version\OSA_project\Emir_Clean\saved_idx"
NUKS_idx = np.load(os.path.join(path, r"NUKS_idxv5.npy"))
mode_lock_idx = np.load(os.path.join(path, r"mode_lock_idxv5.npy"))
movement_idx = np.load(os.path.join(path, r"movement_idxv5.npy"))
vid_movement_idx = np.load(os.path.join(path, r"vid_movement_idxv5.npy"))

In [ ]:
np.random.seed(0)
torch.manual_seed(0)
dataset = []
gt_sigs = []
for i in tqdm(range(len(fused_dataset_train))):
    if(i in NUKS_idx):
        # print("NUKS")
        pass
    elif(i in mode_lock_idx):
        # print('mode lock')
        pass
    elif(i in movement_idx):
        # print('movement')
        pass
    elif(i in vid_movement_idx):
        pass
    else:
        thermal = fused_dataset_train[i]['thermal']
        thermal = utils.highpass_filter(thermal, fs=30, fc=0.1/2, order=6)
        # thermal = signal.savgol_filter(thermal, 41, 3)
        thermal = (thermal - np.mean(thermal))/np.std(thermal)
        dataset.append(thermal)
        gt_sigs.append(np.array([fused_dataset_train[i]['gt_ir'][:,0], fused_dataset_train[i]['gt_OSA'][:,0], fused_dataset_train[i]['gt_CSA'][:,0], fused_dataset_train[i]['gt_radar'][:,0]]))


In [ ]:
len(dataset), len(gt_sigs)

In [ ]:
np.save(r"E:\OSA_project\Emir_Clean\saved_data\thermal_arrv5.npy", dataset)
np.save(r"E:\OSA_project\Emir_Clean\saved_data\gt_arrv5.npy", gt_sigs)


In [ ]:
# for i in tqdm(range(100, len(next_movement_idx))):
#     if(True):
#         data = next_movement_idx[i]
#         thermal = fused_dataset_train[data]['thermal']
#         thermal = utils.highpass_filter(thermal, fs=30, fc=0.1, order=2)
#         thermal = (thermal - np.mean(thermal))/np.std(thermal)
#         dists_min, dists_max, lmin, lmax = utils.movement_detector(thermal, dmin=29, dmax=31, gt_breathing=fused_dataset_train[data]['gt_ir'][:,0], plot=True)
#         plt.plot(t_arr, fused_dataset_train[data]['gt_OSA'][:,0])
#         plt.plot(t_arr, fused_dataset_train[data]['gt_CSA'][:,0], linewidth = 7.0)
#         plt.show()
# for i in tqdm(range(0, len(movement_idx))):
#     thermal = fused_dataset_train[movement_idx[i]]['thermal']
#     thermal = (thermal - np.mean(thermal))/np.std(thermal)
#     print('sample index: ', movement_idx[i])
#     dists_min, dists_max, lmin, lmax = utils.movement_detector(thermal, dmin=29, dmax=31, gt_breathing=fused_dataset_train[movement_idx[i]]['gt_ir'][:,0], plot=True)
#     plt.plot(t_arr, fused_dataset_train[movement_idx[i]]['gt_OSA'][:,0], linewidth=7.0, label='OSA')
#     plt.plot(t_arr, fused_dataset_train[movement_idx[i]]['gt_CSA'][:,0])
#     plt.legend()
#     plt.show()

In [ ]:
import utils

In [ ]:
full_arr = []
prod_arr = []

th_arr = np.linspace(0.1, 0.4, 15)
window_size_arr = np.linspace(20,40, 10, dtype=int)
modes = ['90th', 'max', 'median', 'mean']
percentile = 50

for th in tqdm(th_arr):
    for window_max in tqdm(window_size_arr):
        for window_min in window_size_arr:
            for mode in modes:
                gt_arr = []
                pred_arr = []
                

                for idx in range(len(fused_dataset_train)):
                    if((idx not in NUKS_idx) and (idx not in mode_lock_idx) and (idx not in movement_idx)):
                        pred = utils.predict(fused_dataset_train[idx]['thermal'], dmin=window_min, dmax=window_max, th=th, mode=mode, percentage=percentile, plot=False)

                        gt_OSA = fused_dataset_train[idx]['gt_OSA'][:,0]
                        gt_CSA = fused_dataset_train[idx]['gt_CSA'][:,0]
                        # gt_hp = gt_hypopnea[idx]

                        if((np.mean(gt_OSA) > 0.1) or (np.mean(gt_CSA) > 0.1)):# or (np.mean(gt_hp) > 0.1)):
                            gt_arr.append(1)
                        else:
                            gt_arr.append(0)
                        
                        if(np.mean(pred) > 0.1):
                            pred_arr.append(1)
                        else:
                            pred_arr.append(0)
                        
                        # if(gt_arr[-1] != pred_arr[-1]):
                        #     print(idx)
                
                        # AHI_arr.append(np.sum(pred_arr) / (1/60))
                        # gt_AHI_arr.append(np.sum(gt_arr) / (1/60))

                precision, recall, accuracy, confusion_matrix = utils.get_stats(np.array(pred_arr), np.array(gt_arr))
                # print("Precision: ", precision, "Recall: ", recall, "Accuracy: ", accuracy, "Confusion Matrix: ", confusion_matrix)
                
                prod_arr.append(precision*recall)
                full_arr.append((th, mode, window_min, window_max, recall, precision, accuracy, confusion_matrix))

In [ ]:
for i, data in tqdm(enumerate(fused_dataset_train)):
    # print(i)
    batch = data['thermal']
    thermal = (thermal - np.mean(thermal))/np.std(thermal)
    if(utils.detect_NUKS(thermal) == True):
        NUKS_idx.append(i)

print(100*len(NUKS_idx)/len(fused_dataset_train))
print(len(NUKS_idx))

In [ ]:
def dmax_func(lmax):
    return(int(4*((len(lmax))/10)**(1/2)))

In [ ]:
# idx = apnea_idx[3]
for idx in apnea_idx:
    thermal = fused_dataset_train[idx]['thermal']
    thermal = (thermal - np.mean(thermal))/np.std(thermal)
    pred = utils.predict(thermal, dmin=30, dmax=20, dmin_func=dmax_func, dmax_func=dmax_func, th=0.2, mode='max', percentage=25, interval=None, plot=True)
    gt = fused_dataset_train[idx]["gt_CSA"][:,0]

    print(utils.detect_NUKS(thermal))
    # # print(utils.detect_mode_lock(fused_dataset_train, idx))
    plt.plot(t_arr, gt, label='gt')
    plt.plot(t_arr, pred, label='pred')
    plt.legend()
    plt.show()

In [ ]:
# 1.1) Implement the new motion detection algorithm for thermal
# 1.2) Next steps is to finetune hyperparameters

# 2.1) 

In [ ]:
# 1. Read Kang Paper
# 2. Get familiar with the thermal code
# 3. visualize samples that the algorith gets wrong and change dmax, dmin func accordingly


In [ ]:
power = []
filt_s = []
idx = apnea_idx[15]
radar_data = fused_dataset_train[idx]['radar']
gt_radar = fused_dataset_train[idx]['gt_radar']
gt = fused_dataset_train[idx]["gt_CSA"][:,0]

for i in range(25):
        radar_phase = utils.unwrap_phase(radar_data[:, 0, 0, :, i])
        radar_phase = (radar_phase - np.mean(radar_phase)) / np.std(radar_phase)


        filtered = utils.highpass_filter(radar_phase, order=6, fc=0.1/2, plot=False)
        filtered_x = signal.savgol_filter(filtered, 41, 3)
        filt_s.append(filtered_x)

        f, Pxx_spec = signal.welch(filtered_x, fs, 'flattop', 1024, scaling='spectrum')
        # plt.figure()
        # plt.semilogy(f, np.sqrt(Pxx_spec))
        # plt.xlabel('frequency [Hz]')
        # plt.ylabel('Linear spectrum [V RMS]')
        # plt.show()
        # print(np.sqrt(Pxx_spec.max()))
        power.append(np.sqrt(Pxx_spec.max()))
        # print(i)
        # plt.plot(filtered_x, label='filtered')
        # plt.plot(gt_radar, label='gt')
        # plt.legend()
        # plt.show()
        # print(r"##########################################")
print(np.argmax(power))
plt.figure()
rad_sig = -filt_s[np.argmax(power)]
rad_sig = (rad_sig - np.mean(rad_sig)) / np.std(rad_sig)
plt.plot(t_arr, rad_sig)
plt.plot(t_arr, gt_radar, label='gt')
plt.plot(t_arr, gt)
plt.legend()
plt.show()

In [ ]:
# dmax = int(2*((len(lmax))/17)**(1/2))

In [ ]:
import pickle
path = r"E:\OSA_project\gt_dict.pkl"

with open(path, "rb") as f:
    gt_dict = pickle.load(f)

print(gt_dict.keys())

In [ ]:
signal = np.load(r"E:\OSA_project\signal.npy")
gt_OSA = gt_dict['OSA']
gt_CSA = gt_dict['CSA']
gt_ir = gt_dict['AIR_flow']

In [ ]:
plt.plot(gt_ir)

In [ ]:
import utils

In [ ]:
start = 1800*1
thermal = signal[(start+0):(start+1800)]
thermal = (thermal-np.mean(thermal))/np.std(thermal)
plt.plot(thermal)
plt.plot(utils.highpass_filter(thermal, fs=30, fc=0.1, order=2))
plt.plot(gt_CSA[(start+0):(start+1800)])
plt.plot(gt_OSA[(start+0):(start+1800)])

In [ ]:
def hl_envelopes_idx(s, dmin_num=1, dmax_num=1, dmax_func=None, dmin_func=None, step_size=1, display=False):
    """
    Input :
    s: 1d-array, data signal from which to extract high and low envelopes
    dmin, dmax: int, optional, size of chunks, use this if the size of the input signal is too big
    split: bool, optional, if True, split the signal in half along its mean, might help to generate the envelope in some cases
    Output :
    lmin,lmax : high/low envelope idx of input signal s
    """
    lmin = (np.diff(np.sign(np.diff(s))) > 0).nonzero()[0] + 1 
    lmax = (np.diff(np.sign(np.diff(s))) < 0).nonzero()[0] + 1

    dmin = dmin_num
    dmax = dmax_num

    if(dmax_func is not None):
        dmax = dmax_func(lmax)

    if(dmin_func is not None):
        print('len(lmin): ', len(lmin), '\n')
        dmin = dmin_func(lmin)
    
    if(display == True):
        print('len(lmax): ', len(lmax), '\n')
        print('len(lmin): ', len(lmin), '\n')
        print('step size: ', step_size, '\n')

    lmin = lmin[[max(i-dmin//2,0)+np.argmin(s[lmin[max(i-dmin//2,0):min(i+dmin//2,len(lmin))]]) for i in range(0,len(lmin),step_size)]]
    lmax = lmax[[max(i-dmax//2,0)+np.argmax(s[lmax[max(i-dmax//2,0):min(i+dmax//2,len(lmax))]]) for i in range(0,len(lmax),step_size)]]
    
    return lmin,lmax